## Using CNN and NCM for CIFAR-10 feature extraction and classification

### Summary
CIFAR-10 데이터셋을 분류하기 위해 CNN을 이용하여 데이터셋에 대한 특징을 추출 후 NCM을 이용하여 각 class에 대한 평균 값을 이용하여 분류

<span style="color: #2D3748; background-color:#fff5b1;">Test size를 0.2로 10번 반복 실험한 결과 평균적으로 0.34의 정확도를 보여주고 있고, test 데이터 1개를 분류하는데 0.0011초의 시간이 걸린다.</span>

In [1]:
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from utils.util import Info
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from sklearn.neighbors import NearestCentroid
from utils.Data_Classifier import train, validate, save_checkpoint

c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
class Config(Info):
    def __init__(self):
        super(Info, self).__init__()
        self.device = 'PC'
        self.dataset = 'CIFAR_10'
        self.test_size = 0.2
        self.feature_size = 3072
        self.method = 'CNN'
        self.iter = 10

In [3]:
# cig = Config()
# cig.info()
# cig.print_rutin()

In [4]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

## Load CIFAR-10 Dataset

In [5]:
transform_train = transforms.Compose([transforms.Resize(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                      ])

transform_test = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),                                     
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                     ])

batch_size = 512

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

validationset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
val_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False, num_workers=2)

print(trainset.data.shape)
print(validationset.data.shape)

Files already downloaded and verified
Files already downloaded and verified
(50000, 32, 32, 3)
(10000, 32, 32, 3)


## Fine-tuning Fully Connected Layer

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

epoch = 10
num_class = 10

model = resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_feature = model.fc.in_features
model.fc = nn.Linear(num_feature, num_class)
model.to(device)

feature_extract = True

params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params_to_update, lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

Params to learn:
	 fc.weight
	 fc.bias


In [10]:
best_acc = 0.
train_time = []
val_time = []

for idx in range(epoch):  # loop over the dataset multiple times

    # train for one epoch
    end = time.time()
    train(train_loader, model, criterion, optimizer, idx, device)
    train_time.append(time.time() - end)

    # evaluate on validation set
    end = time.time()
    acc = validate(val_loader, model, criterion, device)
    val_time.append(time.time() - end)

    # remember best acc and save checkpoint
    is_best = acc > best_acc
    best_acc = max(acc, best_acc)

    save_checkpoint({
            'epoch': idx + 1,
            'arch': "ResNet-18",
            'state_dict': model.state_dict(),
            'best_acc1': best_acc,
            'optimizer' : optimizer.state_dict(),
        }, is_best)

    scheduler.step()

train_time = np.array(train_time)
val_time = np.array(val_time)

print('\nFinished Training\n')
print("Total Average Training Time : %.2f ± %.2f" % (train_time.mean(), train_time.std()), "sec")
print("Total Average Validation Time : %.2f ± %.2f" % (val_time.mean(), val_time.std()), "sec")
print("\nTotal Best Validation Accuracy : %.2f" % best_acc.real, "%")

Epoch: [0]
Train: [98/98]	Time 0.3971	Data 0.0558	Loss 1.9159	Acc 67.76
Test: [20/20]	Time 0.4406	Data 0.1264	Loss 0.8629	Acc 77.31
Epoch: [1]
Train: [98/98]	Time 0.4356	Data 0.0571	Loss 0.7673	Acc 77.53
Test: [20/20]	Time 0.4820	Data 0.1441	Loss 0.7883	Acc 76.18
Epoch: [2]
Train: [98/98]	Time 0.4100	Data 0.0553	Loss 0.6762	Acc 78.38
Test: [20/20]	Time 0.4669	Data 0.1295	Loss 0.7598	Acc 76.33
Epoch: [3]
Train: [98/98]	Time 0.4730	Data 0.0793	Loss 0.6269	Acc 79.16
Test: [20/20]	Time 0.4955	Data 0.1477	Loss 0.7059	Acc 76.88
Epoch: [4]
Train: [98/98]	Time 0.4806	Data 0.0820	Loss 0.6351	Acc 79.00
Test: [20/20]	Time 0.6326	Data 0.1505	Loss 0.7608	Acc 76.48
Epoch: [5]
Train: [98/98]	Time 0.4806	Data 0.0975	Loss 0.6394	Acc 79.17
Test: [20/20]	Time 0.4944	Data 0.1518	Loss 0.6506	Acc 78.98
Epoch: [6]
Train: [98/98]	Time 0.4781	Data 0.0836	Loss 0.6824	Acc 78.25
Test: [20/20]	Time 0.4274	Data 0.1401	Loss 0.9444	Acc 73.43
Epoch: [7]
Train: [98/98]	Time 0.3801	Data 0.0533	Loss 0.6610	Acc 78.93
Test

## Fine-tuning NCM instead of Fully Connected Layer

In [6]:
from utils.NCM_Classifier import train, validate, save_checkpoint

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 10

model = resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.to(device)

In [10]:
best_acc = 0.

# train for one epoch
end = time.time()
x_train, y_train = train(train_loader, model, device)

nc = NearestCentroid()
nc.fit(x_train, y_train)
train_time = time.time() - end

# evaluate on validation set
end = time.time()
x_test, y_test = validate(val_loader, model, device)

test_score = nc.score(x_test, y_test)
val_time = time.time() - end

print('\nFinished Training\n')
print("Total Training Time : %.2f" % train_time, "sec")
print("Total Validation Time : %.2f" % val_time, "sec")
print("\nTotal Best Validation Accuracy : %.2f" % (test_score*100), "%")


Finished Training

Total Training Time : 36.65 sec
Total Validation Time : 9.56 sec

Total Best Validation Accuracy : 76.98 %
